In [1]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.vectorstores import DocArrayInMemorySearch
from langchain_openai.embeddings import OpenAIEmbeddings
AMDIRE_PDF_PATH = "test_data/amdire.pdf"

loader = PyPDFLoader(AMDIRE_PDF_PATH)

pages = loader.load_and_split()

embeddings = OpenAIEmbeddings()
vectorstore = DocArrayInMemorySearch.from_documents(pages, embedding=embeddings)


In [7]:
retriever = vectorstore.as_retriever()

retriever.invoke("role of busyness analyst")

[Document(page_content='The AMDiRE Approach 15\nfaces. While we consider the context and the requirements speciﬁcation to\naddress the problem space, the system speciﬁcation addresses the solution\nspace and is the interface to tie in with the design phase.\nFigure 5 shows the artefact types in relation to roles and responsibilities\n(left side) and in relation to milestones (right side) which we use to integrate\nthe model into a process.\nRole ModelProcess ModelProject Scope deﬁned\nSystem SpeciﬁcationacceptedBusiness AnalystRequirementsEngineerSystem ArchitectArchitectureOverviewdeﬁnedRequirementsSpeciﬁcationacceptedSystem VisiondeﬁnedContextSpeciﬁcationacceptedContext Speciﬁcation\nSystem SpeciﬁcationRequirements Speciﬁcation\nFig. 5: Overview of artefacts types, roles, and milestones.\nFor each artefact type, we deﬁne one particular role, which has the respon-\nsibility for an artefact type, independent of other potentially supporting roles\nprovided by the software process model 

In [8]:
from langchain.prompts import ChatPromptTemplate

template = """
Answer the question based on the context below. If you can't 
answer the question, reply "I don't know".

Context: {context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

In [9]:
from langchain_openai import ChatOpenAI
import dotenv
dotenv.load_dotenv()

chat_model = ChatOpenAI(
    model="gpt-3.5-turbo-1106",
    temperature=0
)

In [10]:
from langchain_core.runnables import RunnablePassthrough

chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | chat_model
 | 
)


In [12]:
chain.invoke("What are the main responsibilities of busyness analyst in requirements engineering?")

AIMessage(content='The main responsibilities of a Business Analyst in requirements engineering include having the necessary domain knowledge, such as understanding the business processes, typical stakeholders, and constraints and rules.', response_metadata={'token_usage': {'completion_tokens': 32, 'prompt_tokens': 2854, 'total_tokens': 2886}, 'model_name': 'gpt-3.5-turbo-1106', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-50aaf7c2-7aab-4943-96ba-5fae3be145cb-0')

In [ ]:
chain.stream # gradually shows text 
chain.batch # batch of questions give answers 
